In [ ]:
#Structure de données : DataFrame
import pandas as pd
import numpy as np
%matplotlib inline

var1 = pd.read_csv('../input/housesalesprediction/kc_house_data.csv')
var1['log_sqft_lot'] = np.log(1+var1['sqft_lot'])
var1['log_price'] = np.log(1+var1['price'])

var1.head(2)

var2 = var1[ ['log_sqft_lot', 'floors', 'log_price'] ]
var3 = var1['zipcode']

# create dummy variables
dummies = pd.get_dummies(var3)

merged = pd.concat([var2,dummies],axis='columns')

#drop one column to prevent the dummy variables trap
final = merged.drop([98001], axis='columns')

X = final.drop('log_price', axis='columns')
y = final['log_price']

#80% des données apprentissage
#20% des données pour le test
train_var1 = X[0:17000]
test_var1  = X[17000:]

#Separer les données en X et Y
train_X = train_var1
train_Y = y[0:17000].values

test_X = test_var1.values
test_Y = y[17000:].values

print( train_X.shape, test_X.shape )
print( train_Y.shape, test_Y.shape )

import keras
from keras import layers
from keras.models import Model
from keras.optimizers import SGD

# préciser combien d'attributs en entrée (dans cet exemple: sqft_lot, floors)
inpt = layers.Input( shape=(train_X.shape[1],) , name='entree') 
x    = layers.Dense(200, name='cachee') ( inpt )
actv = layers.Activation('relu') (x)
output = layers.Dense(1, name='sortie') (actv)  

model = Model( inpt, output )

model.summary()

opt = SGD() #choisir le gradient descent comme optimizer pour la mise à jour des poids

model.compile( optimizer=opt, loss='mse')

model.fit( train_X, train_Y, epochs=5 )

predictions = model.predict(test_X)

np.exp(predictions[:20])-1